In [1]:
%matplotlib inline
import numpy as np
import itertools
import logging

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

from config import CFG
from kde_classes import Model, KDE

# Logging setup utilities
from debugging import (
    setup_logger,
    setup_console_handler
)

Welcome to JupyROOT 6.14/04


In [ ]:
setup_logger('KDE', logging.DEBUG)
log_format = '%(asctime)s %(processName)s %(name)s %(levelname)s: '\
             '%(message)s'
setup_console_handler('KDE', logging.DEBUG, log_format)

In [2]:
#mc = np.load(CFG['paths']['mg_mc'])
model = Model('models.sig_E', mc=None, weighting='plotter_wkde')

In [3]:
kde = KDE(model)

In [6]:
result = kde.cross_validate([0.05, 0.11], adaptive=True, pdf_seed=None)
print(result)

[(0.05, 0.11, -1.3331515, 0.)]


In [8]:
pdf_seed = kde.generate_binned_kd([0.05, 0.11])

In [9]:
result = kde.cross_validate([0.05, 0.11], adaptive=True, pdf_seed=pdf_seed)
print(result)

[(0.05, 0.11, -1.333231, 0.)]


In [11]:
kde.model.approx_pdf = pdf_seed
result = kde.cross_validate([0.05, 0.11], adaptive=True, pdf_seed=None)
print(result)

[(0.05, 0.11, -1.3320591, 0.)]


In [ ]:
out_bins = []
for i, key in enumerate(model.vars):
    out_bins.append(np.linspace(model.ranges[i][0],
                                model.ranges[i][1],
                                model.nbins[i]))
    print(key)
coords = np.array(list(itertools.product(*out_bins)))
X, Y = np.meshgrid(*out_bins[:-1], indexing='ij')

In [ ]:
binned_kernel_density = kde.generate_binned_kd([0.06, 0.16])

In [ ]:
pdf_vals = np.asarray([kde.eval_point(binned_kernel_density, coord) for coord in coords])
pdf_vals = pdf_vals.reshape(model.nbins)

In [ ]:
plt.figure(figsize=(8, 8))

plt.pcolormesh(X, Y, pdf_vals)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, pdf_vals)
ax.set_xlabel('x, $\sigma=1$')
ax.set_ylabel('y, $\sigma=2$')

Generate adaptive kde

In [ ]:
adaptive_kernel_density = kde.generate_adaptive_kd([0.5, 0.6])

In [ ]:
adaptive_pdf_vals = np.asarray([kde.eval_point(adaptive_kernel_density, coord) for coord in coords])
adaptive_pdf_vals = adaptive_pdf_vals.reshape(model.nbins)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, adaptive_pdf_vals)
ax.set_xlabel('x, $\sigma=1$')
ax.set_ylabel('y, $\sigma=2$')

Plot mc data

In [ ]:
from dataset import load_and_prepare_data

mc = load_and_prepare_data(CFG['paths']['IC_mc'])

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
#h = ax.hist2d(mc['x'], mc['y'], bins=[100, 100], range=[[-5, 5], [-5, 5]])
h = ax.hist2d(np.log10(mc['sigma_pull_corrected']), mc['logE'], bins=[200, 100])
fig.colorbar(h[3], ax=ax)

In [ ]:
cv_result = kde.cross_validate([0.5, 0.5], adaptive=False)

In [ ]:
cv_result

In [ ]:
result = kde.cross_validate_bandwidths(bandwidths=[[0.5, 0.6], [0.6]], adaptive=True)

In [ ]:
result

In [ ]:
result[result['LLH'] == np.max(result['LLH'])]

In [ ]:
import glob
import numpy as np

from config import CFG
from dataset import load_and_prepare_data
from kde_classes import Model, KDE

mc = np.load(CFG['paths']['mg_mc'])

model = Model('models.multi_gaussian', mc, weighting=None)
kde = KDE(model)

cv_files = glob.glob('output/multi_gaussian/cv_*.npy')
cv_results = np.array([], dtype=kde.cv_result.dtype)

print(kde.cv_result.dtype)
for cv_file in cv_files:
    cv_result = np.loadtxt(cv_file, dtype=kde.cv_result.dtype)
    cv_results = np.append(cv_results, re)

print(cv_results)

In [ ]:
f = np.load('output/sig_psi_E/cv/cv_31.npy')
f.dtype

In [ ]:
import glob
import numpy as np

from config import CFG
from dataset import load_and_prepare_data
from kde_classes import Model, KDE

#mc = np.load('data/multi_gaussian.npy')

model = Model('models.sig_psi_E', mc=None, weighting=None)
kde = KDE(model)

cv_files = glob.glob('output/sig_psi_E/cv/cv_*.npy')
print(kde.cv_result.dtype)
print(cv_files)
cv_results = np.array([], dtype=kde.cv_result.dtype)

for cv_file in cv_files:
    cv_result = np.load(cv_file)
    print(cv_result.dtype)
    cv_results = np.append(cv_results, cv_result)

print(cv_results)

result_max_LLH = cv_results[cv_results['LLH'] == np.max(cv_results['LLH'])]

bandwidth = [result_max_LLH[key] for key in model.bandwidth_vars]

print(bandwidth)

In [ ]:
bandwidth[1]

## Comparison of old vs new cv

In [12]:
import cPickle as pickle

with open('/home/ge56lag/Software/KDE/output/sig_E/pdf/sig_E.pkl', 'rb') as f:
            sig_E = pickle.load(f)
with open('/home/ge56lag/Software/KDE/output/sig_E/old/pdf/sig_E.pkl', 'rb') as f:
            sig_E_old = pickle.load(f)
        
print(sig_E)
print(sig_E_old)

{'pdf_vals': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'bw': [array([0.1], dtype=float32), array([0.17], dtype=float32)], 'coords': array([[-3.09966765,  2.00000132],
       [-3.09966765,  2.06041058],
       [-3.09966765,  2.12081983],
       ...,
       [ 1.14384474,  7.85969877],
       [ 1.14384474,  7.92010802],
       [ 1.14384474,  7.98051727]]), 'vars': ['sigma_pull_corrected', 'log_e'], 'bins': [array([-3.09966765, -3.05680389, -3.01394013, -2.97107637, -2.92821261,
       -2.88534884, -2.84248508, -2.79962132, -2.75675756, -2.7138938 ,
       -2.67103004, -2.62816628, -2.58530251, -2.54243875, -2.49957499,
       -2.45671123, -2.41384747, -2.37098371, -2.32811994, -2.28525618,
       -2.24239242, -2.19952866, -2.1566649 , -2.11380114, -2.07093738,
       -2.02807361, -1.98520985,

In [13]:
result = kde.cross_validate([0.1, 0.17], adaptive=True, pdf_seed=None)
print(result)

result = kde.cross_validate([0.05, 0.11], adaptive=True, pdf_seed=None)
print(result)

[(0.1, 0.17, -1.3308442, 0.)]
[(0.05, 0.11, -1.3320591, 0.)]


In [4]:
import cPickle as pickle

with open('/home/ge56lag/Software/KDE/output/sig_E/pdf/sig_E.pkl', 'rb') as f:
            sig_E = pickle.load(f)
with open('/home/ge56lag/Software/KDE/output/sig_psi_E/pdf/sig_psi_E.pkl', 'rb') as f:
            sig_psi_E = pickle.load(f)

In [5]:
print(sig_E['bw'], sig_psi_E['bw'])

([array([0.05], dtype=float32), array([0.11], dtype=float32)], [array([0.11], dtype=float32), array([0.22], dtype=float32), array([0.25], dtype=float32)])


In [ ]:
plt.figure()

plt.pcolormesh(X, Y, sig_psi_E['pdf_vals'][:, :, 50])

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, sig_psi_E['pdf_vals'][:, :, 10])
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
bg['pdf_vals'][:, :, 0].shape

In [ ]:
eval(str("[2, 3]"))

In [ ]:
model.mc['sigma_pull_corrected']

In [ ]:
model.mc.dtype.names

In [ ]:
from dataset import load_and_prepare_data

mc = np.load('/home/ge56lag/Data/diffuse_mc_wBDT.npy')

In [ ]:
mc['sigma_pull_corrected_mu']

In [ ]:
mc.dtype.names

In [ ]:
mc = np.load('/home/ge56lag/Data/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy')

In [ ]:
mc.dtype.names

In [ ]:
cv = np.load('output/bg/cv/cv_0.npy')

In [ ]:
cv